In [17]:
from time import time
from bs4 import BeautifulSoup
from numpy.lib import utils
import requests
import os
import glob
import pandas as pd
from datetime import datetime
import utils

In [36]:
response = requests.get('https://vnexpress.net/bi-bat-sau-15-nam-troi-chi-dau-4566496.html')
bts = BeautifulSoup(response.content, "html.parser")

In [43]:
for i in bts.find_all('span', attrs ={"class":"location-stamp"}):
    print(i.text)

Nghệ An


In [46]:
# Function read content from link
def read_content(link):
    content_list = []
    response = requests.get(link)   
    bts = BeautifulSoup(response.content, "html.parser")
    
    content = bts.find_all('p', attrs={"class":"Normal"})
    for i in content:
        content_list.append(i.text)
    
    time = bts.find('time')
    if time is None:
        time = "Not defined"
    else:
        time = time.text
    
    location = bts.find('span', attrs={"class":"location-stamp"})
    if location is None:
        location = "Not defined"
    else:
        location = location.text.replace('\r\n', '')
    return ' '.join(content_list), time, location

In [47]:
get_data_from_page('https://vnexpress.net/category/day?cateid=1001007&fromdate=1675382400&todate=1675382400&allcate=1001007')

,Paper,Title_name,Title_link,Title_content,Title_time,Location
0,vnexpress.net,Bị truy tố sau 15 năm trói chị dâu,https://vnexpress.net/bi-bat-sau-15-nam-troi-c...,"Ngày 3/2, VKSND huyện Kỳ Sơn truy tố Xồng Bá X...",Not defined,Nghệ An
1,vnexpress.net,Cựu phó giám đốc CDC Hà Giang bị bắt vì liên q...,https://vnexpress.net/cuu-pho-giam-doc-cdc-ha-...,"Ngày 3/2, bà Dung cùng Hoàng Thị Phượng (45 tu...",Not defined,Not defined
2,vnexpress.net,Đề xuất tăng hạn tuổi phục vụ trong công an,https://vnexpress.net/de-xuat-tang-han-tuoi-ph...,Bộ Công an đang lấy ý kiến đóng góp cho dự thả...,Not defined,Not defined
3,vnexpress.net,Bà lão tạo hiện trường giả bị cướp 250 triệu đồng,https://vnexpress.net/ba-lao-tao-hien-truong-g...,"Ngày 3/2, Công an huyện Châu Phú cho biết đang...",Not defined,An Giang
4,vnexpress.net,Sát hại nhân tình khi hàn gắn bất thành,https://vnexpress.net/sat-hai-nhan-tinh-khi-ha...,"Theo điều tra, Lâm và người phụ nữ hàng xóm 41...",Not defined,Bạc Liêu


In [35]:
read_content('https://vnexpress.net/bi-bat-sau-15-nam-troi-chi-dau-4566496.html')

('Nghệ AnXồng Bá Xử bỏ trốn suốt 15 năm sau khi gây ra vụ án trói và đánh chị dâu và người đàn ông xin ngủ nhờ do nghi có tình cảm. Ngày 3/2, VKSND huyện Kỳ Sơn truy tố Xồng Bá Xử, trú xã Nậm Cắn, huyện Kỳ Sơn về tội Bắt, giữ người trái pháp luật. Xồng Bá Xử sau khi bị bắt. Ảnh: Hoàng Tuấn Theo hồ sơ, ngày 30/8/2006, Xử đến chơi và ngủ tại nhà chị dâu ở địa bàn. Tại đây hôm đó có người đàn ông tên Lồng bị say rượu xin nghỉ nhờ. 4h sáng hôm sau, đúng lúc Lồng nhờ nữ chủ nhà lấy hộ nước uống thì Xừ tỉnh ngủ. Cho rằng hai người đang bày tỏ tình cảm, Xử đấm Lồng đồng thời dùng dây thừng trói cả hai. Xừ tiếp tục dùng thanh củi đánh Lồng. Hai nạn nhân sau đó được một số người dân phát hiện, giải cứu. Công an huyện Kỳ Sơn khởi tố Xử về tội Bắt, giữ người trái pháp luật. Trong thời gian tại ngoại, Xử trốn sang Lào, thay tên và kết hôn với người bản địa. Ngày 11/1, Xừ bị Công an Nghệ An bắt khi vào rừng lấy củi một mình. \nHải Bình\n',
 'Not defined',
 'Not defined')

In [33]:
def get_data_from_page(page_link):
    response = requests.get(page_link)
    bts = BeautifulSoup(response.content, "html.parser")

    paper = []
    title_name = []
    title_link = []
    tilte_content = []
    tilte_time = []
    location = []

    for i in bts.find_all('h3'):
        paper.append('vnexpress.net')
        title_name.append(i.a['title'])
        title_link.append(i.a['href'])
        tilte_content.append(read_content(i.a['href'])[0])
        tilte_time.append(read_content(i.a['href'])[1])
        location.append(read_content(i.a['href'])[2])

    dantri = { 
                "Paper": paper, 
                "Title_name": title_name, 
                "Title_link": title_link, 
                "Title_content": tilte_content, 
                "Title_time": tilte_time, 
                "Location": location
                    }
    return pd.DataFrame(dantri)

In [5]:
def convert_date(date):
    """
    convert string to date: dd/mm/yyyy
    """
    date = date.split('/')
    year = int(date[2])
    month = int(date[1])
    day = int(date[0])
    return datetime(year, month, day), pd.Timestamp(datetime(year, month, day)).quarter, year, month


In [11]:
def dantri_collection():
    start_date = date(2019, 12, 30)
    end_date = date(2019, 12, 31)
    delta = end_date - start_date
    for i in range(delta.days + 1):
        day = start_date + timedelta(days=i)
        day = datetime.strptime(str(day), "%Y-%m-%d").strftime("%d-%m-%Y")
        page_link = 'https://dantri.com.vn/phap-luat/{page_date}.htm'.format(page_date=day)
        df = dantri.get_data_from_page(page_link)
        df.to_csv('/Users/hieudt/VNU/Thesis/Dantri/dantri_{page_date}.csv'.format(page_date=day))
        print(page_link)

In [7]:
def location_process(text):
    text = text.strip()
    text = text.replace(':', '')
    return text

In [8]:
def time_process(text):
    if text != 'Not defined':
        text = text.split(',')[1].split('-')[0]
        text = utils.convert_date(text)
    return text

In [9]:
def dantri_all_processing ():
    dantri = pd.read_csv('/Users/hieudt/VNU/Thesis/Dantri/dantri_collection.csv')
    dantri['Location'] = dantri.apply(lambda row: location_process(row['Location']), axis=1)
    dantri['Month'] = dantri.apply(lambda row: time_process(row['Title_time'])[3], axis=1)
    dantri['Quarter'] = dantri.apply(lambda row: time_process(row['Title_time'])[1], axis=1)
    dantri['Year'] = dantri.apply(lambda row: time_process(row['Title_time'])[2], axis=1)
    dantri = dantri.drop(dantri[dantri['Year'] == 2021].index, axis=0)
    dantri.to_csv('/Users/hieudt/VNU/Thesis/Thesis-Data_Science-main/dantri_all_processing.csv', index=False, encoding='utf-8-sig')

In [10]:
def dantri_result():
    list_of_group_words = utils.word_combination('word_combination.xlsx')
    data = pd.read_csv('dantri_all_processing.csv')
    data['based_index'] = data.apply(lambda row: utils.base_index(list_of_group_words, row['Title_content'])[0], axis=1)
    data['List_out'] = data.apply(lambda row: utils.base_index(list_of_group_words, row['Title_content'])[1], axis=1)
    data = data[data['based_index'] > 0]
    data.to_excel('dantri_result.xlsx', index=False, encoding='utf-8-sig')

In [35]:
read_content('https://dantri.com.vn/phap-luat/ket-cuc-bi-tham-cua-nu-ky-su-doi-nguoi-tinh-tra-no-20221217084429712.htm')

('Ngày 14/7/2016, cảnh sát Singapore nhận một báo cáo về người mất tích. Báo cáo nói rằng Thôi Nhã Kiệt (Cui Yajie), một nữ kỹ sư tới từ Thiên Tân (Trung Quốc) đã không tới chỗ làm từ ngày 12/7/2016. Một số người thấy nữ kỹ sư lần cuối vào lúc 9h15 hôm 11/7/2016 ở khu vực Fusionopolis Walk. Phận thảm của nữ kỹ sư tài năng Là con một trong một gia đình ở thành phố Thiên Tân, Kiệt tốt nghiệp Đại học Southampton (Anh) và tới Singapore để làm việc vào năm 2012.\xa0 Bằng các biện pháp nghiệp vụ, lực lượng cảnh sát hình sự sàng lọc các mối quan hệ của nạn nhân và trích xuất camera trên các phố. Cuối cùng, họ xác định nghi phạm là Leslie Khoo Kwee Hock, một gã đàn ông làm việc tại một tiệm giặt là. Gã đàn ông sinh năm 1968 là người cuối cùng gặp Kiệt. Leslie Khoo Kwee Hock bị bắt giữ ngày 20/6/2016. Nghi phạm 48 tuổi đã dẫn cảnh sát tới nơi hắn giấu thi thể Thôi Nhã Kiệt ở khu vực nhiều cây thuộc đường Lim Chu Kang Lane 8. Tại đây cảnh sát chỉ còn tìm thấy một phần rất nhỏ của thi thể. Kết qu

In [30]:
content_list = []
response = requests.get('https://dantri.com.vn/phap-luat/cuu-tu-lenh-canh-sat-bien-ban-bac-voi-cap-duoi-rut-50-ty-dong-nhu-the-nao-20230202091414993.htm')   
bts = BeautifulSoup(response.content, "html.parser")

In [31]:
time = bts.find('time')
time.text

' Thứ năm, 02/02/2023 - 09:32 '